In [1]:
import json
import os
import pandas as pd
from PIL import Image
import re
import random


In [2]:
base_sqa_dir='/qfs/projects/steel_thread/hora620/DevHub/scientific-instruction-tuning/ScienceQA/data/scienceqa/'
problems = json.load(open(os.path.join(base_sqa_dir, "problems.json")))

In [3]:
problems_T=pd.DataFrame(problems).T

In [16]:
problems['1526']

{'question': 'Which month is the wettest on average in Christchurch?',
 'choices': ['August', 'April', 'May'],
 'answer': 2,
 'hint': 'Use the graph to answer the question below.',
 'image': 'image.png',
 'task': 'closed choice',
 'grade': 'grade3',
 'subject': 'natural science',
 'topic': 'earth-science',
 'category': 'Weather and climate',
 'skill': 'Use climate data to make predictions',
 'lecture': 'Scientists record climate data from places around the world. Precipitation, or rain and snow, is one type of climate data.\nA bar graph can be used to show the average amount of precipitation each month. Months with taller bars have more precipitation on average.',
 'solution': 'To describe the average precipitation trends in Christchurch, look at the graph.\nChoice "Apr" is incorrect.\nChoice "May" is incorrect.\nChoice "Aug" is incorrect.\nMay has an average monthly precipitation of about 70 millimeters. This is higher than in any other month. So, May is the wettest month on average.'

In [5]:
# 7B + C-> "/rcfs/projects/steel_thread/models/LLAVA-7b-pretrain-scitune-333472"

# 7B + CTOM -> "/rcfs/projects/steel_thread/models/LLAVA-7b-pretrain-scitune-333472-v2"

# 13B + CTOM -> "/rcfs/projects/steel_thread/models/LLAVA-7b-pretrain-scitune-333472-v2-13B"

# 13B-2 + CTOM -> "/rcfs/projects/steel_thread/models/LLAVA-2-pretrain-scitune-333472-v2-13B"

base_model_dir='/rcfs/projects/steel_thread/models/LLAVA-7b-pretrain-scitune-333472-v2-13B'
prediction_file='%s/scienceqa/llava_test_QCM-LEPA_prediction.jsonl'%base_model_dir
predictions = [json.loads(line) for line in open(prediction_file)]

results_file='%s/scienceqa/llava_test_QCM-LEPA_prediction_result.jsonl'%base_model_dir
results=json.load(open(results_file,'r'))
print('Accuracy: ',results['acc'])
print('#Corrects: %d/%d'%(results['correct'],results['count']))

results_outputs_file='%s/scienceqa/llava_test_QCM-LEPA_prediction_output.jsonl'%base_model_dir
results_outputs=json.load(open(results_outputs_file,'r'))
results_outputs_0=pd.DataFrame(results_outputs['correct']).set_index('question_id')
results_outputs_0['accuracy_label']='correct'
results_outputs_1=pd.DataFrame(results_outputs['incorrect']).set_index('question_id')
results_outputs_1['accuracy_label']='incorrect'
results_outputs=pd.concat([results_outputs_0,results_outputs_1])

Accuracy:  90.02593727894364
#Corrects: 3818/4241


In [6]:
results_outputs.head()

,parsed_ans,ground_truth,question,pred,is_multimodal,accuracy_label
question_id,,,,,,
4,B,B,Which figure of speech is used in this text?\n...,Assistant: LECTURE: Figures of speech are word...,False,correct
5,B,B,Which of the following could Gordon's test sho...,Assistant: LECTURE: People can use the enginee...,True,correct
11,B,B,What is the name of the colony shown?\nContext...,Assistant: SOLUTION: The colony is New Hampshi...,True,correct
15,A,A,Would you find the word dozen on a dictionary ...,Assistant: LECTURE: Guide words appear on each...,False,correct
23,B,B,Which of these organisms contains matter that ...,Assistant: LECTURE: A food web is a model.\nA ...,True,correct


In [11]:
sample_question="Which month is the wettest on average in Christchurch?"
X=results_outputs[results_outputs['question'].str.match(sample_question)]#.iloc[0]['index']

In [17]:
print(X.iloc[0]['pred'])

Assistant: LECTURE: Scientists record climate data from places around the world. Precipitation, or rain and snow, is one type of climate data. Scientists collect data over many years. They can use this data to calculate the average precipitation for each month. The average precipitation can be used to describe the climate of a location.
A bar graph can be used to show the average amount of precipitation each month. Months with taller bars have more precipitation on average.
SOLUTION: To describe the average precipitation trends in Christchurch, look at the graph.
Choice "Apr" is incorrect.
Choice "May" is incorrect.
Choice "Aug" is incorrect.
May has an average monthly precipitation of about 70 millimeters. This is higher than in any other month. So, May is the wettest month on average.
 The answer is C.


In [ ]:
llava_results_file='./llava/eval/table/results/test_sqa_llava_13b_v0.json'
llava_results=json.load(open(llava_results_file,'r'))
print('LLaVA Accuracy: ',llava_results['acc'])
print('#Corrects: %d/%d'%(llava_results['correct'],llava_results['count']))

In [ ]:
llava_results_outputs=pd.DataFrame(llava_results['outputs'].items(),columns=['question_id','predictions'])#.set_index('question_id')

def get_pred_idx(prediction, choices):
    """
    Get the index (e.g. 2) from the prediction (e.g. 'C')
    """
    options=["A", "B", "C", "D", "E"]
    if prediction in options[:len(choices)]:
        return options.index(prediction)
    else:
        return random.choice(range(len(choices)))
    
def get_pred_label(row):
    pred_text=row['predictions']
    q_id=row['question_id']
    pattern = re.compile(r'The answer is ([A-Z]).')
    res = pattern.findall(pred_text)
    if len(res) == 1:
        answer = res[0]  # 'A', 'B', ...
    else:
        answer = "FAILED"

    return get_pred_idx(answer,problems[q_id]['choices'])

  
llava_results_outputs['gt_answer_index']=llava_results_outputs.apply(lambda x: problems[x['question_id']]['answer'],axis=1)                   
llava_results_outputs['pred_answer_index']=llava_results_outputs.apply(get_pred_label,axis=1)
llava_results_outputs.set_index('question_id',inplace=True)

# llava_results_outputs_1=pd.DataFrame(llava_results['results'].items(),columns=['question_id','gt_answer_index']).set_index('question_id')

# llava_results_outputs=pd.merge(llava_results_outputs_0,llava_results_outputs_1,left_index=True, right_index=True)

llava_results_outputs['accuracy_index']=0
llava_results_outputs.loc[llava_results_outputs['pred_answer_index']==llava_results_outputs['gt_answer_index'],'accuracy_index']=1



llava_results_outputs_with_metadata=pd.merge(llava_results_outputs,problems_T,how='inner',left_index=True,right_index=True)

print("LLaVA Accuracy (Global): ",llava_results_outputs_with_metadata['accuracy_index'].mean())

In [ ]:
llava_results_outputs_with_metadata.head()#loc['4']['predictions']

In [9]:
question_id="1526"
# print("Generated Answer: ",results['outputs'][question_id])
# print("Answer Index: ",results['results'][question_id])
#display(results_outputs.loc[question_id])

problems[question_id]


{'question': 'Which month is the wettest on average in Christchurch?',
 'choices': ['August', 'April', 'May'],
 'answer': 2,
 'hint': 'Use the graph to answer the question below.',
 'image': 'image.png',
 'task': 'closed choice',
 'grade': 'grade3',
 'subject': 'natural science',
 'topic': 'earth-science',
 'category': 'Weather and climate',
 'skill': 'Use climate data to make predictions',
 'lecture': 'Scientists record climate data from places around the world. Precipitation, or rain and snow, is one type of climate data.\nA bar graph can be used to show the average amount of precipitation each month. Months with taller bars have more precipitation on average.',
 'solution': 'To describe the average precipitation trends in Christchurch, look at the graph.\nChoice "Apr" is incorrect.\nChoice "May" is incorrect.\nChoice "Aug" is incorrect.\nMay has an average monthly precipitation of about 70 millimeters. This is higher than in any other month. So, May is the wettest month on average.'

In [ ]:
# test_problems=[]
# for question_id in problems.keys():
    
#     test_problem=problems[question_id]
#     if test_problem['split']=="test":
#         test_problem.setdefault("question_id",question_id)
#         test_problem.setdefault("prediction",results_outputs.loc[question_id]['pred'])
#         test_problem.setdefault("accuracy_label",results_outputs.loc[question_id]['accuracy_label'])
#         test_problems.append(test_problem)

# # ## Optional; saved the javascript array for demo
# # import json
# # with open('test_problems.js', 'w') as fout:
# #     json.dump(test_problems, fout)

In [ ]:
print("Question:\t%s\n"%results_outputs.loc[question_id]['question'])
print("Prediction:\t%s\n"%results_outputs.loc[question_id]['pred'])

print("Answer (parsed)\t%s\n"%results_outputs.loc[question_id]['parsed_ans'])
print("Answer (GT)\t%s\n"%results_outputs.loc[question_id]['ground_truth'])

print("Is Multimodal:\t%s\n"%results_outputs.loc[question_id]['is_multimodal'])
print("Accuracy Label:\t%s\n"%results_outputs.loc[question_id]['accuracy_label'])


In [ ]:
# sel_q="Which trait did Palaeopython have?"
# sel_records=results_outputs_with_metadata[results_outputs_with_metadata['question_x'].str.contains(sel_q)]#.query('question_x==@sel_q')
# sel_records.index=sel_records.index.astype(str)
# sel_records

In [ ]:
sel_q_id='2005'
print(sel_records.loc[[sel_q_id]]['question_x'].iloc[0])

print(sel_records.loc[[sel_q_id]]['choices'].iloc[0])
print(sel_records.loc[[sel_q_id]]['lecture'].iloc[0])

print(sel_records.loc[[sel_q_id]]['pred'].iloc[0])
print(sel_records.loc[[sel_q_id]]['accuracy_label'].iloc[0])


print("\n\n## LLaVA")
print("",llava_results['outputs'][sel_q_id])
image = Image.open(os.path.join(base_sqa_dir, 'images/test/%s/image.png'%sel_q_id)).convert('RGB')
image

In [10]:
image = Image.open(os.path.join(base_sqa_dir, 'images/val/%s/image.png'%question_id)).convert('RGB')
image

FileNotFoundError: [Errno 2] No such file or directory: '/qfs/projects/steel_thread/hora620/DevHub/scientific-instruction-tuning/ScienceQA/data/scienceqa/images/val/1526/image.png'

## LLaMA-SciTune Performance Table

In [7]:

results_outputs_with_metadata=pd.merge(results_outputs,problems_T,how='inner',left_index=True,right_index=True)

In [8]:
results_outputs_with_metadata['accuracy_index']=0
results_outputs_with_metadata.loc[results_outputs_with_metadata['accuracy_label']=="correct",'accuracy_index']=1

In [9]:
print("Accuracy (Global): ",results_outputs_with_metadata['accuracy_index'].mean())

Accuracy (Global):  0.821032775288847


In [10]:
print("Accuracy (Global): ",results_outputs_with_metadata.groupby('is_multimodal')['accuracy_index'].mean())

Accuracy (Global):  is_multimodal
False    0.881745
True     0.754090
Name: accuracy_index, dtype: float64


In [11]:
print("Accuracy (Global): ",results_outputs_with_metadata.groupby('subject')['accuracy_index'].mean())

Accuracy (Global):  subject
language science    0.835455
natural science     0.838366
social science      0.759280
Name: accuracy_index, dtype: float64


In [12]:
print("Accuracy (Global): ",results_outputs_with_metadata.groupby('grade')['accuracy_index'].mean())

Accuracy (Global):  grade
grade1     0.705882
grade10    0.815126
grade11    0.828571
grade12    0.906977
grade2     0.819113
grade3     0.846395
grade4     0.855869
grade5     0.824534
grade6     0.799564
grade7     0.803887
grade8     0.739884
grade9     0.942623
Name: accuracy_index, dtype: float64


In [13]:
sel_grades=['grade1','grade2','grade3','grade4','grade5','grade6']
print("Accuracy (G1-6): ",results_outputs_with_metadata.groupby('grade')['accuracy_index'].mean()[sel_grades].mean())

sel_grades=['grade7','grade8','grade9','grade10','grade11','grade12']
print("Accuracy (G7-12): ",results_outputs_with_metadata.groupby('grade')['accuracy_index'].mean()[sel_grades].mean())

Accuracy (G1-6):  0.8085596065159736
Accuracy (G7-12):  0.8395114154759918


In [14]:
results_outputs_with_metadata['is_context']=~results_outputs_with_metadata['question_x'].str.contains('Context: N/A')
print("Accuracy (Context): ",results_outputs_with_metadata.groupby('is_context')['accuracy_index'].mean())

Accuracy (Context):  is_context
False    0.824146
True     0.817693
Name: accuracy_index, dtype: float64


In [ ]:
llava_results_outputs_with_metadata.head()

## LLaMA SciTune vs. LLaVA

In [ ]:
#sel_cols=['predictions','gt_answer_index','pred_answer_index',]
llava_scitune_results_outputs=pd.merge(results_outputs_with_metadata[['accuracy_index']],llava_results_outputs_with_metadata[['accuracy_index']],left_index=True,right_index=True,how='inner')

In [ ]:
scitune_top=llava_scitune_results_outputs[(llava_scitune_results_outputs['accuracy_index_x']==1) & (llava_scitune_results_outputs['accuracy_index_y']==0)]
scitune_top_with_metadata=pd.merge(scitune_top,problems_T,how='inner',left_index=True,right_index=True)

In [ ]:
llava_top=llava_scitune_results_outputs[(llava_scitune_results_outputs['accuracy_index_x']==0) & (llava_scitune_results_outputs['accuracy_index_y']==1)]
llava_top_with_metadata=pd.merge(llava_top,problems_T,how='inner',left_index=True,right_index=True)

In [ ]:
problems_T.head()

In [ ]:
scitune_top_with_metadata.groupby('skill').size().sort_values(ascending=False)#head()

In [ ]:
llava_top_with_metadata.groupby('skill').size().sort_values(ascending=False)#head()